In [5]:
# All imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
import copy
import statsmodels.api as sm
import matplotlib
matplotlib.rcParams.update({'figure.max_open_warning':0})
matplotlib.use("agg")
# Add path to transformer code in the syspath in order to recognize the modules and import them
import sys
sys.path.insert(0, os.path.abspath('../code/LSTM_TRANSF/Trasformer/'))
from transformer.decoder import Decoder
from transformer.multihead_attention import MultiHeadAttention
from transformer.positional_encoding import PositionalEncoding
from transformer.pointerwise_feedforward import PointerwiseFeedforward
from transformer.encoder_decoder import EncoderDecoder
from transformer.encoder import Encoder
from transformer.encoder_layer import EncoderLayer
from transformer.decoder_layer import DecoderLayer
from transformer.noam_opt import NoamOpt
from transformer.batch import subsequent_mask
from transformer.flow import run_epoch

transformer.decoder.Decoder

In [ ]:


class GeneratorTS(nn.Module):
    "Define standard linear + softmax generation step."

    def __init__(self, d_model, vocab):
        super(GeneratorTS, self).__init__()
        self.proj = nn.Linear(d_model, vocab, )

    def forward(self, x):
        return self.proj(x)


def make_model(src_vocab, tgt_vocab, N=6,
               d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadAttention(h, d_model)
    ff = PointerwiseFeedforward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn),
                             c(ff), dropout), N),
        nn.Sequential(NewEmbed(src_vocab, d_model), c(position)),
        nn.Sequential(NewEmbed(src_vocab, d_model), c(position)),
        GeneratorTS(d_model, tgt_vocab))

    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model


class NewEmbed(nn.Module):
    def __init__(self, vocab, d_model):
        super(NewEmbed, self).__init__()
        # lut => lookup table
        self.lut = nn.Linear(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)


class Batch:
    "Object for holding a batch of data with mask during training."

    def __init__(self, src, trg=None):
        self.src = src
        # self.src_mask = Variable(torch.ones(src.shape[:-1]))
        # self.src_mask2 = (src != pad).unsqueeze(-2)
        self.src_mask = torch.ones(src.shape[:-1]).unsqueeze(-2)

        if trg is not None:
            self.trg_old = torch.cat((torch.ones((trg.shape[0], 1, trg.shape[-1])) * -1, trg), 1)
            self.trg = self.trg_old[:, :-1]
            self.trg_y = self.trg_old[:, 1:]
            self.trg_mask = self.make_std_mask(self.trg, -2)
            self.ntokens = trg.shape[0] * trg.shape[1]
        if cuda:
            self.src = self.src.cuda()
            self.src_mask = self.src_mask.cuda()
            self.trg = self.trg.cuda()
            self.trg_y = self.trg_y.cuda()
            self.trg_mask = self.trg_mask.cuda()

    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt[:, :, 0] != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & (
            subsequent_mask(tgt.size(-2)).type_as(tgt_mask.data))
        return tgt_mask


def make_batch(src, tgt, batch_size, permu=True):
    i = 0

    perm = np.random.permutation(src.shape[0])
    if permu:
        src = src[perm, :]
        tgt = tgt[perm, :]
    while i < src.shape[0]:
        yield Batch(torch.from_numpy(src[i:i + batch_size, :]),
                    torch.from_numpy(tgt[i:i + batch_size, :]))
        i += batch_size


class SimpleLossCompute:
    "A simple loss compute and train function."

    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt

    def __call__(self, x, y, norm):
        if self.opt is not None:
            self.opt.optimizer.zero_grad()

        x = self.generator(x)
        loss = self.criterion(x, y[:, :, 0:1])

        if self.opt is not None:
            loss.backward()
            self.opt.step()

        return loss.data.item() * norm


def greedy_decode(model, src, src_mask, max_len, start_symbol, ys_meta):
    memory = model.encode(src.unsqueeze(0), src_mask.unsqueeze(0))
    ys = torch.ones(1, 1, src.shape[-1]).fill_(start_symbol).type_as(src.data)
    for i in range(max_len - 1):
        out = model.decode(memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data))
        val_out = model.generator(out)
        # _, next_word = torch.max(prob, dim=1)
        # next_word = next_word.data[0]
        ys = torch.cat([ys, torch.cat((val_out, ys_meta[i, 1:].unsqueeze(0)), dim=1).unsqueeze(0)], dim=1)
    return ys


def greedy_decode_batch(model, src, src_mask, max_len, start_symbol, ys_meta):
    memory = model.encode(src, src_mask)
    ys = torch.ones(src.shape[0], 1, src.shape[-1]).fill_(start_symbol).type_as(src.data)
    for i in range(max_len - 1):
        out = model.decode(memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data))
        val_out = model.generator(out)[:, -1]
        # _, next_word = torch.max(prob, dim=1)
        # next_word = next_word.data[0]
        ys = torch.cat([ys, torch.cat((val_out.unsqueeze(-1), ys_meta[:, i, 1:].unsqueeze(1)), dim=2)], dim=1)
    return ys










##########

pretrained = False
cuda = True
cuda = cuda and torch.cuda.is_available()

d_total = pd.read_csv("Dati Negozi/ALBIG_elaborato.csv",delimiter=",", parse_dates=[0], index_col=0)

#d_2008 = pd.read_csv("ERCOT/2008.csv", delimiter=";", decimal=",", parse_dates=[0], index_col=0)
#d_2009 = pd.read_csv("ERCOT/2009.csv", delimiter=";", decimal=",", parse_dates=[0], index_col=0)
#d_2010 = pd.read_csv("ERCOT/2010.csv", delimiter=";", decimal=",", parse_dates=[0], index_col=0)
#d_2011 = pd.read_csv("ERCOT/2011.csv", delimiter=";", decimal=",", parse_dates=[0], index_col=0)
#d_2012 = pd.read_csv("ERCOT/2012.csv", delimiter=";", decimal=",", parse_dates=[0], index_col=0)
#d_2013 = pd.read_csv("ERCOT/2013.csv", delimiter=";", decimal=",", parse_dates=[0], index_col=0)
#d_2014 = pd.read_csv("ERCOT/2014.csv", delimiter=";", decimal=",", parse_dates=[0], index_col=0)
d_2015 = pd.read_csv("ERCOT/2015.csv", delimiter=";", decimal=",", parse_dates=[0], index_col=0)
print(d_2015.shape)
print(d_20XX.shape)

#train = pd.concat([d_2008, d_2009, d_2010, d_2011, d_2012, d_2013, d_2014, d_2015])['ERCOT']
train = pd.concat([d_2012, d_2013, d_2014, d_2015])['ERCOT'] #C'ERA ANCHE  all'inizio

test = d_2016['ERCOT']

train = train.resample('H').mean().fillna(method='pad')
test = test.resample('H').mean().fillna(method='pad')

tr_weekDay = pd.get_dummies(train.index.dayofweek, dtype=np.float32)
tr_weekDay.index = train.index
te_weekDay = pd.get_dummies(test.index.dayofweek, dtype=np.float32)
te_weekDay.index = test.index

tr_year = train.index.year
te_year = test.index.year

tr_month = train.index.month
te_month = test.index.month

tr_hour = train.index.hour
te_hour = test.index.hour

mean = train.values.mean()
std  = train.values.std()
tr_df = pd.DataFrame({'val': (train.values -mean)/std, 'year': tr_year.values, 'hour': tr_hour.values / 24,
                      'ts': ((train.index - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")).values},
                     index=train.index.values, dtype=np.float32, columns=['val', 'year', 'hour', 'ts'])
te_df = pd.DataFrame({'val': (test.values -mean)/ std, 'year': te_year.values, 'hour': te_hour.values / 24,
                      'ts': ((test.index - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")).values},
                     index=test.index.values, dtype=np.float32, columns=['val', 'year', 'hour', 'ts'])

tr_df = pd.concat([tr_df, tr_weekDay], axis=1)
te_df = pd.concat([te_df, te_weekDay], axis=1)


#atr = [x for x in range(11) if x != 3]
atr = [0, 2, 4, 5, 6, 7, 8, 9, 10]
#atr = [0]
inp_tr = np.stack([tr_df.shift(i).values for i in range(12, 36)], axis=1)[36:, -1::-1, atr].copy()
out_tr = np.stack([tr_df.shift(i).values for i in range(12)], axis=1)[36:, -1::-1, atr].copy()

inp_te = np.stack([te_df.shift(i).values for i in range(12, 36)], axis=1)[36:, -1::-1, atr].copy()
out_te = np.stack([te_df.shift(i).values for i in range(12)], axis=1)[36:, -1::-1, atr].copy()

criterion = torch.nn.MSELoss()
model = make_model(inp_te.shape[-1], 1, N=6, d_model=512, h=8)
if cuda:
    model.cuda()
    criterion = criterion.cuda()
model_opt = NoamOpt(512, 0.1, 2000,
                    torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
# model_opt=torch.optim.Adam(model.parameters(),lr=0.001)

best_model_loss=np.inf
if not pretrained:
    for epoch in range(100):    #dovrebbe essere il for lunghissimo che non smette di allenare
        model.train()           # alle 16.25 del 9/2/20 siamo a 78 cicli di allenamento ogni 10 epoche FINE: 00.33 del 10/2/20
        run_epoch(make_batch(inp_tr, out_tr, 200), model,
                  SimpleLossCompute(model.generator, criterion, model_opt))
        model.eval()
        eval_loss = run_epoch(make_batch(inp_te, out_te, 30), model,
                              SimpleLossCompute(model.generator, criterion, None))
        print(eval_loss)
        if epoch % 1 == 0:
            if eval_loss<best_model_loss:
                torch.save(model.state_dict(), "ercot_best_loss.pth")
                best_model_loss=eval_loss
                with open("best_loss",'w') as f1:
                    f1.write("epoch, loss\n %05i,%10.8f"%(epoch,eval_loss))
            f = open("eval loss.csv", 'a')
            f.write('%04i,%010.8f\n' % (epoch, eval_loss))
            f.close()

            val_out = []
            trg_out = []
            for b in make_batch(inp_te, out_te, 30, False):
                out = model.forward(b.src, b.trg, b.src_mask, b.trg_mask)
                val_out.append(model.generator(out)[:, 0].cpu().data.numpy())
                trg_out.append(b.trg_y[:, 0, 0].cpu().data.numpy())

            val = np.concatenate(val_out, axis=0)
            trg = np.concatenate(trg_out, axis=0)

            x = list(range(1000))

            val_plt = val[x]
            trg_plt = trg[x]
            plt.figure()
            plt.plot(x, trg_plt, label='Ground truth')
            plt.plot(x, val_plt, label='1d forecast')
            plt.legend()
            plt.savefig("plots_ercot/plot_epoch_%03i.png" % (epoch))

if pretrained:
    model.load_state_dict(torch.load("ercot_ckp_195.pth"))
model.eval()
if cuda:
    model.cuda()
val_out = []
trg_out = []
preds_out = []
for b in make_batch(inp_te, out_te, 30, False):
    out = model.forward(b.src, b.trg, b.src_mask, b.trg_mask)
    val_out.append(model.generator(out)[:, 0].cpu().data.numpy())
    trg_out.append(b.trg_y[:, :, 0].cpu().data.numpy())
    # v = greedy_decode(model, b.src[k], b.src_mask[k], max_len=13, start_symbol=-1,ys_meta=b.trg_y[k])
    v = greedy_decode_batch(model, b.src, b.src_mask, max_len=13, start_symbol=-1, ys_meta=b.trg_y)
    vv = v.detach().cpu().numpy()
    preds_out.append(vv[:, 1:, :])
val = np.concatenate(val_out, axis=0)
trg = np.concatenate(trg_out, axis=0)
preds = np.concatenate(preds_out, axis=0)

x = list(range(300))

val_plt = val[x]
trg_plt = trg[x,0]

plt.figure()
plt.plot(x, trg_plt, label='Ground truth')
plt.plot(x, val_plt, label='1h forecast')
plt.legend()
plt.title("1h Forecast")
plt.savefig("plot_epoch_last_1h.png")

tt = test.values[25:][:val.shape[0]]
mae = sm.tools.eval_measures.meanabs(((val[:, 0] * std)+mean).astype(int), tt)
rmse = sm.tools.eval_measures.rmse(((val[:, 0] * std)+mean).astype(int), tt)

print("MAE  1h: %10.4f" % (mae))
print("RMSE 1h: %10.4f" % (rmse))


plt.figure()
plt.plot(x, trg[x,-1], label='Ground truth')
plt.plot(x, preds[x,-1,0], label='12h forecast')
plt.legend()
plt.title("12h Forecast")
plt.savefig("plot_epoch_last_12h.png")

tt = test.values[36:][:preds.shape[0]]
mae = sm.tools.eval_measures.meanabs(((preds[:,-1,0] * std)+mean).astype(int), tt)
rmse = sm.tools.eval_measures.rmse(((preds[:,-1,0] * std)+mean).astype(int), tt)

print("MAE  1h: %10.4f" % (mae))
print("RMSE 1h: %10.4f" % (rmse))
